# Formation 2022  

##  Initiation à l’analyse de données Minion pour l'analyse de métagénome viraux

Created by J. Orjuela (DIADE-IRD), D. Filloux (PHIM-CIRAD) and A. Comte (PHIM-IRD) and from a formation by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) given in Septembre 2021

Septembre 2022

# TP3 : ASSIGNATION TAXONOMIQUE DES READS

Taxonomic assignment is the process of assigning an Operational Taxonomic Unit (OTUs, that is, groups of related individuals) to sequences, that can be reads or contigs. To assign an OTU to a sequence it is compared against a database, but this comparison can be done in different ways. The comparison database in this assignment process must be constructed using complete genomes. There are many programs for doing taxonomic mapping, almost all of them follows one of the next strategies:


- BLAST: Using BLAST or DIAMOND, these mappers search for the most likely hit for each sequence within a database of genomes (i.e. mapping). This strategy is slow.

- K-mers: A genome database is broken into pieces of length k, so as to be able to search for unique pieces by taxonomic group, from lowest common ancestor (LCA), passing through phylum to species. Then, the algorithm breaks the query sequence (reads, contigs) into pieces of length k, look for where these are placed within the tree and make the classification with the most probable position.

- Markers: They look for markers of a database made a priori in the sequences to be classified and assign the taxonomy depending on the hits obtained.

https://carpentries-incubator.github.io/metagenomics/06-taxonomic/index.html

## 1. Use Diamond for taxonomic assignation:

### 1.1. Download Genomic viral bank

In [1]:
# create working repository
mkdir ~/work/SG-ONT-2022/ASSIGNATION/DIAMOND

mkdir: cannot create directory ‘/home/jovyan/work/SG-ONT-2022/ASSIGNATION/DIAMOND’: File exists


: 1

In [2]:
# go inside repository
cd ~/work/SG-ONT-2022/ASSIGNATION/DIAMOND
pwd

/home/jovyan/work/SG-ONT-2022/ASSIGNATION/DIAMOND


In [3]:
# refseq viral database pre-dowloaded from ncbi (https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/)
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/viral.protein.faa

--2022-07-28 13:37:37--  https://itrop.ird.fr/ont-training-2022/viral.protein.faa
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘viral.protein.faa’ not modified on server. Omitting download.



The database you use will determine the result you get for your data.

You can customise it by adding organism to the fasta file used.

Imagine you are searching for a lineage that was recently discovered and it is not part of the available databases. Would you find it?

### 1.2. Create Diamond Database

In [4]:
diamond makedb --in viral.protein.faa -d viral

diamond v2.0.15.153 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: viral.protein.faa
Opening the database file...  [0s]
Loading sequences...  [0.635s]
Masking sequences...  [1.624s]
Writing sequences...  [0.127s]
Hashing sequences...  [0.055s]
Loading sequences...  [0s]
Writing trailer...  [0.007s]
Closing the input file...  [0s]
Closing the database file...  [0s]

Database sequences  591092
  Database letters  141537030
     Database hash  53675924d8042f04d5477877c942426d
        Total time  2.452000s


### 1.3. Lauch Diamond

In [7]:
diamond blastx --quiet -d viral.dmnd --outfmt 6 stitle qtitle pident length mismatch gapopen qstart qend sstart send evalue bitscore -q ../../CLEANING/reads_vs_ananas_unmapped.fastq -o diamond-matches.csv

In [9]:
# observer les hits dans la database qui ont eu le plus de correspondance dans les données:
awk -F '\t' '{print $1}' diamond-matches.csv | sort | uniq -c | sort -n | tail -20

   1434 YP_009352885.1 coat protein [Arracacha virus V]
   1515 YP_009664759.1 coat protein [Actinidia virus A]
   1529 YP_009389466.1 coat protein [Grapevine virus K]
   1545 YP_009505636.1 capsid protein [Grapevine virus D]
   1553 YP_009551970.1 Coat protein [Grapevine virus J]
   2168 NP_619662.1 putative replicase [Grapevine virus A]
   2235 YP_009465945.1 replicase [Grapevine virus I]
   2282 YP_009551946.1 replicase [Grapevine virus G]
   2284 YP_009552539.1 replicase [Grapevine virus G]
   2290 YP_009352883.1 replicase [Arracacha virus V]
   2314 YP_009552718.1 replicase [Blackberry virus A]
   2330 YP_009551905.1 putative replicase [Grapevine virus H]
   2334 YP_006590065.1 replicase [Grapevine virus F]
   2455 YP_009664756.1 putative replicase, partial [Actinidia virus A]
   2537 YP_002117775.1 replicase [Grapevine virus E]
   2582 YP_009551967.1 Replicase [Grapevine virus J]
   2702 YP_004935358.1 ORF1 gene product [Actinidia virus B]
   2719 NP_619654.1 putative replicase [

Observe the csv file results and comment it.

## 2. Use KRAKEN2 for taxonomic assignation:

Kraken is a taxonomic sequence classifier that assigns taxonomic labels to DNA sequences. Kraken examines the k-mers within a query sequence and uses the information within those k-mers to query a database. That database maps k-mers to the lowest common ancestor (LCA) of all genomes known to contain a given k-mer.

In [19]:
kraken2 --help

Usage: kraken2 [options] <filename(s)>

Options:
  --db NAME               Name for Kraken 2 DB
                          (default: none)
  --threads NUM           Number of threads (default: 1)
  --quick                 Quick operation (use first hit or hits)
  --unclassified-out FILENAME
                          Print unclassified sequences to filename
  --classified-out FILENAME
                          Print classified sequences to filename
  --output FILENAME       Print output to filename (default: stdout); "-" will
                          suppress normal output
  --confidence FLOAT      Confidence score threshold (default: 0.0); must be
                          in [0, 1].
  --minimum-base-quality NUM
                          Minimum base quality used in classification (def: 0,
                          only effective with FASTQ input).
  --report FILENAME       Print a report with aggregrate counts/clade to file
  --use-mpa-style         With --report, format report output

### 2.1. Download a viral database

For this TP we will download a pre-made simplified kraken database.

MiniKraken DB_8GB (6.0 GB): A pre-built 8 GB database constructed from complete bacterial, archaeal, and viral genomes in RefSeq (as of Oct. 18, 2017). This can be used by users without the computational resources needed to build a Kraken database. This contains around 5% of kmers from the original standard database. 
It can be found here: https://ccb.jhu.edu/software/kraken/

You can build your own custom Database (see https://github.com/DerrickWood/kraken2/wiki/Manual). However, it take a lot of ressources and time.


In [20]:
# create working repository
mkdir ~/work/SG-ONT-2022/ASSIGNATION/KRAKEN

mkdir: cannot create directory ‘/home/jovyan/work/SG-ONT-2022/ASSIGNATION/KRAKEN’: File exists


: 1

In [21]:
cd ~/work/SG-ONT-2022/ASSIGNATION/KRAKEN

In [22]:
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/minikraken2_v2_8GB_201904.tgz

--2022-07-28 15:07:43--  https://itrop.ird.fr/ont-training-2022/minikraken2_v2_8GB_201904.tgz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5935990636 (5.5G) [application/x-gzip]
Saving to: ‘minikraken2_v2_8GB_201904.tgz’

minikraken2_v2_8GB_ 100%[===================>]   5.53G  55.6MB/s    in 1m 45s  

2022-07-28 15:09:32 (53.9 MB/s) - ‘minikraken2_v2_8GB_201904.tgz’ saved [5935990636/5935990636]

FINISHED --2022-07-28 15:09:32--
Total wall clock time: 1m 49s
Downloaded: 1 files, 5.5G in 1m 45s (53.9 MB/s)


In [23]:
#uncompress the database
tar zxvf minikraken2_v2_8GB_201904.tgz

minikraken2_v2_8GB_201904_UPDATE/
minikraken2_v2_8GB_201904_UPDATE/taxo.k2d
minikraken2_v2_8GB_201904_UPDATE/opts.k2d
minikraken2_v2_8GB_201904_UPDATE/database100mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/database150mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/database200mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/hash.k2d


In [27]:
# Inspect the database content
kraken2-inspect --db minikraken2_v2_8GB_201904_UPDATE/ | head -15

# Database options: nucleotide db, k = 35, l = 31
# Spaced mask = 11111111111111111111111111111111111111001100110011001100110011
# Toggle mask = 1110001101111110001010001100010000100111000110110101101000101101
# Total taxonomy nodes: 21114
# Table size: 1399930605
# Table capacity: 2000000000
# Min clear hash value = 14207118309059100672
100.00	1399930605	619083	R	1	root
 98.96	1385354760	72724	R1	131567	  cellular organisms
 83.11	1163416379	1025603	D	2	    Bacteria
 42.38	593288710	1473535	P	1224	      Proteobacteria
 20.38	285373440	759477	C	1236	        Gammaproteobacteria
  5.69	79638023	3551	O	72274	          Pseudomonadales
  4.72	66079245	52184	F	135621	            Pseudomonadaceae
  4.64	64952993	11121851	G	286	              Pseudomonas


### 2.2. run Kraken

In [33]:
kraken2 --db minikraken2_v2_8GB_201904_UPDATE/ ../../CLEANING/reads_vs_ananas_unmapped.fastq --report report.txt --report-minimizer-data --> output_kraken

Loading database information... done.
405689 sequences (173.94 Mbp) processed in 8.686s (2802.5 Kseq/m, 1201.58 Mbp/m).
  203682 sequences classified (50.21%)
  202007 sequences unclassified (49.79%)


**Standard Kraken Output Format**

Each sequence (or sequence pair, in the case of paired reads) classified by Kraken 2 results in a single line of output. Kraken 2's output lines contain five tab-delimited fields; from left to right, they are:

- "C"/"U": a one letter code indicating that the sequence was either classified or unclassified.

- The sequence ID, obtained from the FASTA/FASTQ header.

- The taxonomy ID Kraken 2 used to label the sequence; this is 0 if the sequence is unclassified.

- The length of the sequence in bp. In the case of paired read data, this will be a string containing the lengths of the two sequences in bp, separated by a pipe character, e.g. "98|94".

- A space-delimited list indicating the LCA mapping of each k-mer in the sequence(s). For example, "562:13 561:4 A:31 0:1 562:3" would indicate that:

        - the first 13 k-mers mapped to taxonomy ID #562
        - the next 4 k-mers mapped to taxonomy ID #561
        - the next 31 k-mers contained an ambiguous nucleotide
        - the next k-mer was not in the database
        - the last 3 k-mers mapped to taxonomy ID #562


In [34]:
head output_kraken

C	799ec77c-6555-4b9f-99a3-e58c9fbc1265	1491	335	0:217 1491:4 0:21 1491:2 0:6 1491:5 0:16 1491:4 2:5 0:20 9606:1
U	37c0c305-d935-4b3b-b336-24b4c4c8021d	0	332	0:56 9606:2 0:240
U	02df7f95-9bbe-4b55-9c8b-78955e3d9210	0	208	0:174
U	b86266e6-4b84-4ed6-abde-302e336f6c24	0	429	0:53 9606:5 0:337
U	b7f946d2-7f1d-492c-b187-3ebc0770a15c	0	292	0:222 131567:2 0:34
U	c2615778-aa7c-4906-8c53-75cd9fa196f9	0	605	0:571
C	b2388cec-c33d-4a6b-948f-4cb151194e5f	1491	417	0:42 9606:1 0:230 1491:1 0:7 1491:2 0:14 1491:5 0:39 1491:1 1239:3 0:38
U	26f48c03-b0cd-485e-8e6e-6cd4d9be7b74	0	245	0:211
U	515b1cfe-bfc5-4595-966f-ecf1c09a0b12	0	595	0:561
U	97e06d49-a952-4adf-b223-8a721c1b48b7	0	253	0:33 9606:2 0:184


**Report format output**

The format with the --report-minimizer-data flag, then, is similar to that described in [Sample Report Output Format], but slightly different. The fields in this new format, from left-to-right, are:

- 1. Percentage of fragments covered by the clade rooted at this taxon
- 2. Number of fragments covered by the clade rooted at this taxon
- 3. Number of fragments assigned directly to this taxon
- 4. Number of minimizers in read data associated with this taxon (new)
- 5. An estimate of the number of distinct minimizers in read data associated with this taxon (new)
- 6. A rank code, indicating (U)nclassified, (R)oot, (D)omain, (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies. Taxa that are not at any of these 10 ranks have a rank code that is formed by using the rank code of the closest ancestor rank with a number indicating the distance from that rank. E.g., "G2" is a rank code indicating a taxon is between genus and species and the grandparent taxon is at the genus rank.
- 7. NCBI taxonomic ID number
- 8. Indented scientific name


In [35]:
head -10 report.txt

 49.79	202007	202007	0	0	U	0	unclassified
 50.21	203682	28	947662	67684	R	1	root
 49.90	202450	2948	939753	67479	R1	131567	  cellular organisms
 25.40	103035	1620	377485	12155	D	2	    Bacteria
 24.08	97708	36	285275	3974	D1	1783272	      Terrabacteria group
 23.78	96459	58	274867	1165	P	1239	        Firmicutes
 23.72	96240	2	270330	429	C	186801	          Clostridia
 23.72	96236	15	270287	403	O	186802	            Clostridiales
 23.71	96193	1	270102	278	F	31979	              Clostridiaceae
 23.71	96192	20	270080	261	G	1485	                Clostridium


## 3. Use KAIJU for taxonomic assignation:

Kaiju is a program for the taxonomic classification of high-throughput sequencing reads, e.g., Illumina or Roche/454, from whole-genome sequencing of metagenomic DNA. Reads are directly assigned to taxa using the NCBI taxonomy and a reference database of protein sequences from microbial and viral genomes.

Kaiju can be also used via a web server. https://kaiju.binf.ku.dk/server

In [37]:
# create working repository
mkdir ~/work/SG-ONT-2022/ASSIGNATION/KAIJU

In [39]:
cd ~/work/SG-ONT-2022/ASSIGNATION/KAIJU

### 3.1 Create Kaiju viruses database

In [40]:
kaiju-makedb -s viruses

.listing                [ <=>                ]   1.76K  --.-KB/s    in 0s      
2022-07-28 15:23:14 URL: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz [1800] -> ".listing" [1]
taxdump.tar.gz      100%[===================>]  55.90M  21.2MB/s    in 2.6s    
2022-07-28 15:23:17 URL: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz [58613025] -> "taxdump.tar.gz" [1]
Extracting taxdump.tar.gz
Extracting protein sequences from downloaded files
Creating Borrows-Wheeler transform
# infilename= viruses/kaiju_db_viruses.faa
# outfilename= viruses/kaiju_db_viruses
# Alphabet= ACDEFGHIKLMNPQRSTVWY
# nThreads= 5
# length= 0.000000
# checkpoint= 3
# caseSens=OFF
# revComp=OFF
# term= *
# revsort=OFF
# help=OFF
Sequences read time = 0.628739s
SLEN 145872300
NSEQ 586242
ALPH *ACDEFGHIKLMNPQRSTVWY
SA NCHECK=1
Sorting done,  time = 77.812180s
Creating FM-Index
# filenm= viruses/kaiju_db_viruses
# removecmd= NULL (null)
# help=OFF
Reading BWT from file viruses/kaiju_db_viruses.bwt ... DONE

### 3.2 Launch Kaiju (a little bit long to run)

In [45]:
kaiju --help

kaiju: invalid option -- '-'
Kaiju 1.9.0
Copyright 2015-2022 Peter Menzel, Anders Krogh
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>

Usage:
   kaiju -t nodes.dmp -f kaiju_db.fmi -i reads.fastq [-j reads2.fastq]

Mandatory arguments:
   -t FILENAME   Name of nodes.dmp file
   -f FILENAME   Name of database (.fmi) file
   -i FILENAME   Name of input file containing reads in FASTA or FASTQ format

Optional arguments:
   -j FILENAME   Name of second input file for paired-end reads
   -o FILENAME   Name of output file. If not specified, output will be printed to STDOUT
   -z INT        Number of parallel threads for classification (default: 1)
   -a STRING     Run mode, either "mem"  or "greedy" (default: greedy)
   -e INT        Number of mismatches allowed in Greedy mode (default: 3)
   -m INT        Minimum match length (default: 11)
   -s INT        Minimum match score in Greedy mode (default: 65)
   -E FLOAT      Minimum E-value in Greedy mode (default

: 1

In [49]:
kaiju -t nodes.dmp -f viruses/kaiju_db_viruses.fmi -i ~/work/SG-ONT-2022/CLEANING/reads_vs_ananas_unmapped.fastq -v -o kaiju.out

Parameters: 
  run mode: Greedy
  minimum match length: 11
  seed length: 7
  minimum blosum62 score for matches: 65
  minimum E-value: 0.01
  max number of mismatches within a match: 3
  input file 1: /home/jovyan/work/SG-ONT-2022/CLEANING/reads_vs_ananas_unmapped.fastq
  output file: kaiju.out
15:34:20 Reading database
 Reading taxonomic tree from file nodes.dmp
 Reading index from file viruses/kaiju_db_viruses.fmi
15:34:21 Start classification using 1 threads.
15:40:34 Finished.


**output format**

Kaiju will print one line for each read or read pair. The default output format contains three columns separated by tabs. Using the option -v enables the verbose output, which will print additional columns:

- either C or U, indicating whether the read is classified or unclassified.
- name of the read
- NCBI taxon identifier of the assigned taxon
- the length or score of the best match used for classification
- the taxon identifiers of all database sequences with the best match
- the accession numbers of all database sequences with the best match
- matching fragment sequence(s)

In [50]:
head kaiju.out

U	799ec77c-6555-4b9f-99a3-e58c9fbc1265	0
U	37c0c305-d935-4b3b-b336-24b4c4c8021d	0
U	02df7f95-9bbe-4b55-9c8b-78955e3d9210	0
U	b86266e6-4b84-4ed6-abde-302e336f6c24	0
U	b7f946d2-7f1d-492c-b187-3ebc0770a15c	0
C	c2615778-aa7c-4906-8c53-75cd9fa196f9	136234	440	136234,	YP_009664796.1,	RTITFNTGGRKTMYGVYEGEEVRSYLNALTFRGEYISNVEGNRTDSATFSVSSDGILSVSVNGTLLKNDLVPSPPTVFSKNLEYLSNIEK,
U	b2388cec-c33d-4a6b-948f-4cb151194e5f	0
U	26f48c03-b0cd-485e-8e6e-6cd4d9be7b74	0
U	515b1cfe-bfc5-4595-966f-ecf1c09a0b12	0
U	97e06d49-a952-4adf-b223-8a721c1b48b7	0


### 3.3. Adding taxa names to output file

In [53]:
kaiju-addTaxonNames -t nodes.dmp -n names.dmp -i kaiju.out -o kaiju.names.out

In [54]:
head kaiju.names.out

U	799ec77c-6555-4b9f-99a3-e58c9fbc1265	0
U	37c0c305-d935-4b3b-b336-24b4c4c8021d	0
U	02df7f95-9bbe-4b55-9c8b-78955e3d9210	0
U	b86266e6-4b84-4ed6-abde-302e336f6c24	0
U	b7f946d2-7f1d-492c-b187-3ebc0770a15c	0
C	c2615778-aa7c-4906-8c53-75cd9fa196f9	136234	440	136234,	YP_009664796.1,	RTITFNTGGRKTMYGVYEGEEVRSYLNALTFRGEYISNVEGNRTDSATFSVSSDGILSVSVNGTLLKNDLVPSPPTVFSKNLEYLSNIEK,	Pineapple mealybug wilt-associated virus 2
U	b2388cec-c33d-4a6b-948f-4cb151194e5f	0
U	26f48c03-b0cd-485e-8e6e-6cd4d9be7b74	0
U	515b1cfe-bfc5-4595-966f-ecf1c09a0b12	0
U	97e06d49-a952-4adf-b223-8a721c1b48b7	0


### 3.3 Creating input file for Krona

In [47]:
kaiju2krona -t nodes.dmp -n names.dmp -i kaiju.out -o kaiju.out.krona

In [48]:
ktImportText -o kaiju.out.html kaiju.out.krona

Writing kaiju.out.html...


**Obeserve the results**

Now open the HTML file by clicking on it on the left menu.

If you have an error : "Javascript must be enabled to view this page", please click on "trust HTML".

What can you see on this Krona?